In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss
from pandas_datareader import DataReader
from datetime import datetime

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [3]:
facebook = DataReader('FB',  'yahoo', datetime(2016,4,1), datetime(2019,9,1));
facebook.reset_index(inplace=True,drop=False)
facebook.set_index('Date')
facebook['OpenTmr'] = facebook['Open'].shift(-1)
facebook['OpenClose']= (facebook['Open']+ facebook['Close'])/2
facebook['HighLow'] = (facebook['High']+ facebook['Low'])/2
facebook['OCHL'] = (facebook['Open']+ facebook['Close']+facebook['High']+ facebook['Low'])/4
facebook.head()

,Date,High,Low,Open,Close,Volume,Adj Close,OpenTmr,OpenClose,HighLow,OCHL
0,2016-04-01,116.169998,113.349998,113.750000,116.059998,24760500,116.059998,114.070000,114.904999,114.759998,114.832499
1,2016-04-04,114.550003,111.610001,114.070000,112.550003,48487900,112.550003,112.110001,113.310001,113.080002,113.195002
2,2016-04-05,113.320000,111.900002,112.110001,112.220001,22962400,112.220001,112.470001,112.165001,112.610001,112.387501
3,2016-04-06,113.809998,112.419998,112.470001,113.709999,20814600,113.709999,113.790001,113.090000,113.114998,113.102499
4,2016-04-07,114.889999,113.070000,113.790001,113.639999,20714500,113.639999,114.250000,113.715000,113.980000,113.847500


In [4]:
N = facebook.shape[0] # total num days
num_boot = 300# total num bootstrap
T= 250 # start day
window = 200 # training period window

In [5]:
df = facebook[['Close']]
df.reset_index(level=0, inplace=True)
df.columns=['ds','y']

X1 = df.y.ewm(span=12, adjust=False).mean()
X2 = df.y.ewm(span=26, adjust=False).mean()
X = X1 - X2
Y =facebook['OpenTmr'][-(N-(T+1)):].values
#X = np.column_stack([np.ones((T,1)),X])
#print(X)
#print(Y.shape)
#movAverage1= (movAvg) 

In [6]:
def MACD(X, Y):
    
    T = X.shape[0]
    #print(T)
    #mu = (facebook['Open '].mean(),facebook['Open '].mean(),facebook['Open '].mean()) 
    #cov = [[1,0.75,-0.35],[0.75,1,0.9],[-0.35,0.9,1]]
    #F = np.random.multivariate_normal(mu,cov,T)
    #Sample for Y,X
    X = np.column_stack([np.ones((T,1)),X])
    #T = X.shape[0]
    N = X.shape
    #beta = np.array([0.56,2.53,2.05,1.78])
    #beta.shape=(N[1],1)
    #Y =X@beta+np.random.normal(0,1,(T,1))
    #Y=facebook['Open '].values
    #print(T)
    invXX = np.linalg.inv(X.transpose()@X)
    beta_hat = invXX@X.transpose()@Y
    y_hat = X@beta_hat
    
    residuals = Y-y_hat
    sigma2 = (1/T)*residuals.transpose()@residuals

    sigma = np.sqrt(sigma2)

    #variance - covariance of beta_hat
    varcov_beta_hat = (sigma2)*invXX
    std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

    R_square = 1-(residuals.transpose()@residuals)/(T*np.var(Y))

    adj_R_square = 1-(1-R_square)*(T-1)/(T - N[1])

    #Testing Coefficents:beta_i
    #Null Hypotesis

    t_stat = (beta_hat.transpose()-0)/std_beta_hat
    p_val = 1-ss.norm.cdf(t_stat)
    #print(p_val)
    #Test of joint significance
    F_stat= (beta_hat.transpose()@np.linalg.inv(varcov_beta_hat)@beta_hat/N[1])/(residuals.transpose()@residuals/(T-N[1]))
    p_val_F= 1 - ss.f.cdf(F_stat,N[1]-1, T-N[1])
    
    return beta_hat,y_hat

In [7]:
def bootstrap():
    T = 250
    #print(T)
    N = X.shape[0]
    #print(N)
    
    yhat_macd = np.zeros(N-(T+1))
    window = 200
    num_boost = 300 # increase
    
    for t in range(T+1,N):
        X_train = df.y.ewm(span=5, adjust=False).mean()[t-window:t-1]
        #X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        Y_train = facebook[['OpenTmr']][t-window:t-1].values
        #print(X_train.shape)
        #print(Y_train.shape)
        X_pred = df.y.ewm(span=5, adjust=False).mean()[t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        yhat_train = MACD(X_train , Y_train)[1]
        res_train = Y_train - yhat_train
       
        y_pred_all = np.zeros(num_boost)
       
        for i in range (0,num_boost):
            #err = np.random.choice(res_train,(window-1, ),replace = True)
            err = res_train
            y_bstr = yhat_train + err
            beta_bstr = MACD(X_train,y_bstr)[0]
            #print(X_pred.shape)
            #print(beta_bstr.shape)
 
            y_pred_bstr = X_pred@beta_bstr
            y_pred_all[i] = y_pred_bstr
           
        y_pred_macd = y_pred_all.mean()
       
        yhat_macd[t-(T+1)] = y_pred_macd
   
   
    rmse_macd = np.sqrt(np.mean((Y[27:-1] - yhat_macd[27:-1])**2))
   
    return yhat_macd,rmse_macd

In [8]:
yhat_macd,rmse_macd = bootstrap()

In [9]:
rmse_macd

5.1444098494264665

In [10]:
yhat_macd.shape

(610,)